In [1]:
from os import environ

# environ['HF_HOME'] = '/Volumes/DATA/hf_home'
environ["HF_HOME"] = "/mnt/workspaces/hf_home"
environ["HTTP_PROXY"] = "http://clash.internal.moqi.ai:7890"
environ["HTTPS_PROXY"] = "http://clash.internal.moqi.ai:7890"
environ["OPENAI_API_BASE"] = "http://10.1.3.28:8990/v1"
environ["OPENAI_API_KEY"] = "sk-"


from retrieval_qa_benchmark.models import *
from retrieval_qa_benchmark.datasets import *
from retrieval_qa_benchmark.transforms import *
from retrieval_qa_benchmark.protocols import *
from retrieval_qa_benchmark.utils.registry import REGISTRY
from retrieval_qa_benchmark.utils.profiler import PROFILER

print(str(REGISTRY))

[nltk_data] Downloading package words to /home/fangruil/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/fangruil/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/fangruil/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/fangruil/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/fangruil/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Datasets:
	hotpot_qa
	mmlu
	wikihop
Transforms:
	base
	Faiss
	ElasticBM25
	MyScale
	RRFHybrid
LLMs:
	remote-llm
	gpt35
	chatgpt35
Evaluators:
	mcma
	mcsa


/home/fangruil/anaconda3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:127: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/fangruil/anaconda3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:127: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [2]:
import yaml
from retrieval_qa_benchmark.utils.config import load

config = load(open("../config/mmlu-llama2-remote-retrieval-myscale.yaml"))
evaluator = REGISTRY.Evaluators["mcsa"].from_config(config)
print(evaluator.transform)

Converting dataset...: 100%|██████████| 324/324 [00:00<00:00, 5564.72it/s]
2023-08-15 01:53:16.883 | INFO     | retrieval_qa_benchmark.transforms.search.myscale:__init__:34 - connecting to myscale backend...
2023-08-15 01:53:16.945 | INFO     | retrieval_qa_benchmark.transforms.search.myscale:__init__:41 - load mpnet model...


{'0': ContextWithMyScale(children=(None, None), sep_chr='\n', num_selected=5, context_template='{title} | {paragraph}', msc_host='10.1.2.118', msc_port=8123, msc_user='admin', msc_pass='moqi.ai', two_staged=True, kw_topk=10, num_filtered=100, embedding_name='paraphrase-multilingual-mpnet-base-v2', table_name='default.Wikipedia')}
entry_id='0' chain={'0': ContextWithMyScale(children=(None, None), sep_chr='\n', num_selected=5, context_template='{title} | {paragraph}', msc_host='10.1.2.118', msc_port=8123, msc_user='admin', msc_pass='moqi.ai', two_staged=True, kw_topk=10, num_filtered=100, embedding_name='paraphrase-multilingual-mpnet-base-v2', table_name='default.Wikipedia')}


In [3]:
import json
from tqdm import tqdm

data = []
for i in tqdm(range(len(evaluator.dataset))):
    data.append(evaluator.transform(evaluator.dataset[i]))
with open("old_aligned.jsonl", "w") as f:
    f.write("\n".join([json.dumps(d.model_dump()) for d in data]))
# evaluator.dataset.eval_set = evaluator.dataset.eval_set[:5]
data[0]

100%|██████████| 324/324 [01:10<00:00,  4.62it/s]


QARecord(id='4e8db4a928333356588d2b89477e5d5bcb95532cd8afa355456ac94e13fd1849', question='Unlike most other early civilizations, Minoan culture shows little evidence of:', answer='conspicuous consumption by elites.', type='MCSA', context=['Minoan civilization | The Bronze Age began on Crete around 3200BC. The Early Bronze Age (3500 to 2100BC) has been described as indicating a "promise of greatness" in light of later developments on the island. In the late third millenniumBC, several locations on the island developed into centers of commerce and handiwork, enabling the upper classes to exercise leadership and expand their influence. It is possible that the original hierarchies of the local elites were replaced by monarchies, a precondition for the palaces. Pottery typical of the Korakou culture was discovered in Crete from the Early Minoan Period.', 'Ancient history | Minoan civilisation emerged around 3000 BC on the island of Crete, where towns emerged on the coast. Trade was importan

In [4]:
print(evaluator.llm.convert_record(data[0]))

The following are multiple choice questions (with answers) with context:

Context:
[1] Minoan civilization | The Bronze Age began on Crete around 3200BC. The Early Bronze Age (3500 to 2100BC) has been described as indicating a "promise of greatness" in light of later developments on the island. In the late third millenniumBC, several locations on the island developed into centers of commerce and handiwork, enabling the upper classes to exercise leadership and expand their influence. It is possible that the original hierarchies of the local elites were replaced by monarchies, a precondition for the palaces. Pottery typical of the Korakou culture was discovered in Crete from the Early Minoan Period.
[2] Ancient history | Minoan civilisation emerged around 3000 BC on the island of Crete, where towns emerged on the coast. Trade was important in Minoan civilisation, with artifacts from the Minoans discovered in Egypt, Syria, Cyprus, and Greece. Large palaces grew up on Crete, decorated with

In [5]:
print(str(PROFILER))
PROFILER.clear()

	======== Profile Result ========
transform.base.profile                            	Count:     648	AvgTime:  107.63 ms
dataset.mmlu.profile                              	Count:     324	AvgTime:    0.01 ms
database.MyScaleSearcher.retrieve.profile         	Count:     324	AvgTime:  186.35 ms
database.MyScaleSearcher.search.profile           	Count:     324	AvgTime:  215.05 ms
transform.MyScale.profile                         	Count:     324	AvgTime:  215.23 ms
transform.TransformChain.profile                  	Count:     324	AvgTime:  215.34 ms



In [ ]:
matched = evaluator()

In [ ]:
print(str(PROFILER))

In [ ]:
with open("results.with-retrieval.test.jsonl", "w") as f:
    f.write("\n".join([r.model_dump_json() for r in matched[1]]))

In [ ]:
for dname, dset in REGISTRY.Datasets.items():
    print(f"Loading {dname}...")
    dset.build()

In [ ]:
from retrieval_qa_benchmark.schema import QAPrediction

mismatched = [pred for pred in matched[1] if pred.matched]

In [ ]:
with open("results.test.jsonl", "w") as f:
    f.write("\n".join([r.model_dump_json() for r in mismatched]))

In [6]:
import json

with open("old_aligned.jsonl") as f1:
    old_result = [json.loads(l) for l in f1.readlines()]

with open("new.jsonl") as f2:
    new_result = [json.loads(l) for l in f2.readlines()]

In [7]:
from hashlib import sha256

cnt = 0
for r1, r2 in zip(old_result, new_result):
    c1 = set(map(lambda x: sha256(x.encode("utf-8")).hexdigest(), r1["context"]))
    c2 = set(map(lambda x: sha256(x.encode("utf-8")).hexdigest(), r2["context"]))
    cnt += len(c2.intersection(c1))
print(cnt / len(new_result) / 5)

0.1993827160493827
